In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import feather
from bs4 import BeautifulSoup
import os
pd.set_option('display.max_columns', None)

In [ ]:
chromedriver = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(chromedriver)

In [ ]:
# Setup Function
def selector_tools():
    selector1 = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="all_lineups-2-man"]/div[1]/div'))
            );
    selector1.click()
        
    selector2 = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="share_on_lineups-2-man"]'))
            );
    selector2.click()
        
    selector3 = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="commands_lineups-2-man"]/div[4]/button[2]'))
            );
    selector3.click()
        
    selector4 = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="share"]/p/input'))
            );
    selector4.click()
        
    selector5 = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-content"]/p[1]/strong/a'))
            );
    selector5.click()

In [ ]:

def scrape_all_players(driver):
    
    main_url = 'http://www.basketball-reference.com/players/s/'
    driver.get(main_url)
    player_list_p = driver.find_elements_by_xpath('//th//a')
    
    def returnthis():
        href = []
        for item in player_list_p:
            href.append(item.get_attribute('href').replace('.html', '/lineups/2016'))
        return href
    
    for z in returnthis():
        driver.get(z);

        url = driver.current_url
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page,"lxml")
        
        check_me = soup.findAll('tr')
        
        if check_me == []:
            continue
        else:
            selector_tools()
            url = driver.current_url
            response = requests.get(url)
            page = response.text
            soup = BeautifulSoup(page,"lxml")
            
            table_headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
            data_rows = soup.findAll('tr')[2:]
        
            player_data = [[td.getText() for td in data_rows[i].findAll('td')] for i in range(len(data_rows))]
            player_data_02 = []
        
            for i in range(len(data_rows)):
                 player_row = []
        
            for td in data_rows[i].findAll('td'):        
                player_row.append(td.getText())        
        
            player_data_02.append(player_row)
            df = pd.DataFrame(player_data, columns=table_headers[1:])
            df = df[:-1]
            
            playername = soup.find('h1').getText()
            playername = playername.split(' 2')[0]
            df['playername'] = playername
            if os.path.isfile('my_2016nbafile18.feather') == False:
                path = 'my_2016nbafile18.feather'
                feather.write_dataframe(df, path)
            else:
                ogdf = feather.read_dataframe('my_2016nbafile18.feather')
                df = ogdf.append(df, ignore_index=True)
                path = 'my_2016nbafile18.feather'
                feather.write_dataframe(df, path)
        
                
scrape_all_players(driver)
